# CSCN8010 Final Project - Energy Estimator

## Group #8
* Eris Leksi
* Erica Holden
* Reham Abuarquob

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from util.Vectorizer import Vectorizer

df = pd.read_csv('./data/alpaca_llama3_70b_server.csv')

# Drop unnecessary column
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])

y = df['energy_consumption_llm']


In [10]:
vectorizer = Vectorizer(df['prompt'])
vectorizer.model.train(df['prompt'], total_examples=len(df['prompt']), epochs=10)

df = vectorizer.update_dataframe_with_prompt_vector(df, 'prompt')
X_vec = df['prompt_vector']
print(df.head())

X_train, X_test, y_train, y_test = train_test_split(X_vec, y, test_size=0.2, random_state=42)
model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
# Calculate accuracy as the percentage of predictions within 10% of actual values
accuracy = (abs(y_test - y_pred) / y_test < 0.1).mean()
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {model.score(X_test, y_test)}')
print(f'Accuracy: {accuracy}')


   index  model_name                           created_at  total_duration  \
0      0  llama3:70b  2024-05-23 09:15:00.187929369+00:00     94879729212   
1      0  llama3:70b  2024-05-23 09:15:18.245447446+00:00     14599356392   
2      0  llama3:70b  2024-05-23 09:18:31.047288630+00:00    189692748218   
3      0  llama3:70b  2024-05-23 09:21:57.558292165+00:00    202650658347   
4      0  llama3:70b  2024-05-23 09:23:29.453819226+00:00     88047573776   

   load_duration  prompt_token_length  prompt_duration  response_token_length  \
0     8831648220                   18       3094979000                    296   
1         317320                   13       2270133000                     46   
2         571421                   13       2253547000                    506   
3         362015                   13       3620154000                    447   
4         421890                   18       2993127000                    306   

   response_duration                              

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, the experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:prompt_vector: object

In [ ]:
# Save the model and vectorizer for future use
import joblib
joblib.dump(model, 'xgboost_model.pkl')
joblib.dump(vectorizer, 'text_vectorizer.pkl')